In [2]:
from datetime import datetime
from transformation_handler import *
import pandas as pd
import numpy as np
from lookups import *

In [ ]:
tup = fetch_data(DataSources.SHELTER_NORFOLK, limit=1000000)

In [4]:
df = tup[1]

In [15]:
import warnings

In [16]:
norfolkk = df.copy()
norfolkk['years_old'] = pd.to_numeric(norfolkk['years_old'])
norfolkk['months_old'] = pd.to_numeric(norfolkk['months_old'])
norfolkk['intake_date'] =  pd.to_datetime(norfolkk['intake_date'])
norfolkk['outcome_date'] =  pd.to_datetime(norfolkk['outcome_date'])
age_in_days = np.where(
    (~norfolkk['years_old'].isna()) | (~norfolkk['months_old'].isna()),
    (norfolkk['years_old'].fillna(0) * 12 + norfolkk['months_old'].fillna(0)) * 30,
    np.nan
)
# The following line is resolving the warning message from the line after it
warnings.filterwarnings("ignore", category=RuntimeWarning, module="pandas.core.arrays.timedeltas")
norfolkk['date_of_birth'] = norfolkk['intake_date'] - pd.to_timedelta(age_in_days, unit='D')
norfolkk.drop(['years_old','months_old'],axis=1, inplace=True)
norfolkk['date_of_birth'] =  pd.to_datetime(norfolkk['date_of_birth'])
norfolkk.rename(columns={'animal_type':'type','primary_breed': 'breed', 'primary_color':'color'},inplace=True)
norfolkk.drop_duplicates(inplace=True)
norfolkk['type'] = np.where(~norfolkk['type'].isin(['Cat', 'Dog']), 'Other', norfolkk['type'])
data_columns = ['breed', 'color']
norfolkk[data_columns] = norfolkk[data_columns].apply(lambda x: x.str.title())
norfolkk['color'] = norfolkk['color'].replace({'Bl |Blk ': 'Black ', 'Brn |Br ' : 'Brown ', 'Org':'Orange', 'Sl ':'Silver '},regex=True)
norfolkk['sex'].replace({'NULL': 'Unknown', 'Intact Male': 'Male', 'Intact Female': 'Female', 'Neutered Male': 'Neutered', 'Spayed Female': 'Spayed'}, inplace=True)
norfolkk.reset_index(drop=True, inplace=True)
norfolkk.intake_type.replace({'Owner Surrendered':'Owner Surrender','Confiscated':'Confiscate'},inplace=True)
norfolkk['outcome_type'] = norfolkk['outcome_type'].fillna('Pending')
norfolkk.outcome_type.replace({'Euthanized':'Euthanize','Disposal of Deceased Pet':'Disposal','Unassisted Death':'Died'},inplace=True)
norfolkk['animal_id'] = norfolkk['animal_id'].apply(lambda x: f'NOR-{x}')
norfolkk['region'] = 'Norfolkk'

In [10]:
norfolkk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27872 entries, 0 to 27871
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   animal_id      27872 non-null  object        
 1   type           27872 non-null  object        
 2   breed          27872 non-null  object        
 3   color          27872 non-null  object        
 4   sex            27872 non-null  object        
 5   intake_date    27872 non-null  datetime64[ns]
 6   outcome_date   27872 non-null  datetime64[ns]
 7   intake_type    27872 non-null  object        
 8   outcome_type   27872 non-null  object        
 9   date_of_birth  25053 non-null  datetime64[ns]
 10  region         27872 non-null  object        
dtypes: datetime64[ns](3), object(8)
memory usage: 2.3+ MB


In [12]:
norfolkk.tail()

,animal_id,type,breed,color,sex,intake_date,outcome_date,intake_type,outcome_type,date_of_birth,region
27867,NOR-A070756,Cat,Domestic Sh,Buff,Male,2023-10-05,2023-10-05,Stray,Foster,2023-09-15 12:00:00,Norfolkk
27868,NOR-A070756,Cat,Domestic Sh,Buff,Male,2023-10-07,2023-10-07,Foster,Euthanize,2023-09-17 12:00:00,Norfolkk
27869,NOR-A070759,Cat,Domestic Sh,Brown Tabby,Female,2023-10-05,2023-10-05,Stray,Euthanize,NaT,Norfolkk
27870,NOR-A070780,Dog,Labrador Retr,Black,Male,2023-10-07,2023-10-10,Owner Surrender,Foster,2023-03-11 00:00:00,Norfolkk
27871,NOR-A070810,Other,Raccoon,Black,Unknown,2023-10-10,2023-10-10,Stray,Euthanize,2022-10-15 00:00:00,Norfolkk


In [13]:
df.tail()

,animal_id,animal_type,primary_breed,primary_color,sex,years_old,months_old,intake_date,outcome_date,intake_type,outcome_type
27902,A070756,Cat,DOMESTIC SH,BUFF,Male,NaN,0.65,2023-10-05T00:00:00.000,2023-10-05T00:00:00.000,Stray,Foster
27903,A070756,Cat,DOMESTIC SH,BUFF,Male,NaN,0.65,2023-10-07T00:00:00.000,2023-10-07T00:00:00.000,Foster,Euthanized
27904,A070759,Cat,DOMESTIC SH,BRN TABBY,Female,NaN,NaN,2023-10-05T00:00:00.000,2023-10-05T00:00:00.000,Stray,Euthanized
27905,A070780,Dog,LABRADOR RETR,BLACK,Male,NaN,7,2023-10-07T00:00:00.000,2023-10-10T00:00:00.000,Owner Surrendered,Foster
27906,A070810,Other,RACCOON,BLACK,Unknown,1,NaN,2023-10-10T00:00:00.000,2023-10-10T00:00:00.000,Stray,Euthanized


In [160]:
norfolkk.type.value_counts()

Cat      14021
Dog      12128
Other     1728
Name: type, dtype: int64

In [165]:
norfolkk.breed.value_counts()

Domestic Sh      12184
Pit Bull          4069
Domestic Mh        997
Labrador Retr      898
Germ Shepherd      634
                 ...  
Havana Brown         1
Owl                  1
Newfoundland         1
Turkish Van          1
Californian          1
Name: breed, Length: 238, dtype: int64

In [173]:
norfolkk.color.value_counts()

Black             7856
Brown Tabby       2802
White             2524
Gray              2395
Tan               1595
Brown             1574
Tricolor          1156
Gray Tabby        1053
Orange Tabby       858
Brown Brindle      832
Black Tabby        589
Red                447
Torbie             431
Tortie             402
Blue               401
Calico             359
Buff               241
Fawn               222
Black Brindle      207
Orange             202
Cream              188
Dil Calico         156
Chocolate          142
Blue Brindle       129
Yellow             124
Sable              109
Black Smoke         88
Silver              85
Seal Pt             84
Green               71
Lynx Pt             70
Red Brindle         67
Blue Merle          64
Dil Tortie          59
Brown Merle         49
Gold                46
Slvr Tabby          38
Pink                33
Flame Pt            31
Crm Tabby           15
Apricot             15
Lilac Pt            12
Choc Pt             11
Red Merle  

In [176]:
norfolkk.sex.value_counts()

Neutered    10336
Spayed       9487
Male         3887
Female       3239
Unknown       928
Name: sex, dtype: int64

In [190]:
norfolkk.intake_type.value_counts()

Stray                               12592
Foster                               7291
Owner Surrender                      4961
Confiscate                           1314
Transfer                              611
Disposal Request of Deceased Pet      538
Return                                309
Euthanasia Request                    237
Temporary                              24
Name: intake_type, dtype: int64

In [199]:
norfolkk.outcome_type.value_counts()

Adoption                      9963
Foster                        7383
Return to Owner               4526
Euthanize                     3401
Transfer                      1660
Disposal                       610
Died                           216
Missing                         90
Returned to Native Habitat      27
Pending                          1
Name: outcome_type, dtype: int64